In [3]:
# Write a program to construct a Bayesian network considering medical data. Use this model to 
# demonstrate the diagnosis of heart patients using standard Heart Disease Data Set

In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:
ht=pd.read_csv('heart.csv')

In [6]:
ht.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [7]:
ht.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [8]:
ht.isnull().sum().sum()

0

In [9]:
from sklearn.model_selection import train_test_split
ht_train,ht_test = train_test_split(ht,test_size = 0.2,random_state = 2)

In [10]:
ht_train.reset_index(drop = True,inplace = True)
ht_test.reset_index(drop = True,inplace = True)

In [11]:
pip install pgmpy

In [13]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

model = BayesianNetwork([('age', 'trestbps'), ('age', 'fbs'), ('sex', 'trestbps'),
                       ('exang', 'trestbps'),('trestbps','target'),('fbs','target'),
                      ('target','restecg'),('target','thalach'),('target','chol')])

In [14]:
model.fit(ht_train, estimator=MaximumLikelihoodEstimator)

In [15]:
print(model.get_cpds('age'))

+---------+------------+
| age(29) | 0.00487805 |
+---------+------------+
| age(34) | 0.00365854 |
+---------+------------+
| age(35) | 0.0121951  |
+---------+------------+
| age(37) | 0.00365854 |
+---------+------------+
| age(38) | 0.0109756  |
+---------+------------+
| age(39) | 0.0109756  |
+---------+------------+
| age(40) | 0.0121951  |
+---------+------------+
| age(41) | 0.0341463  |
+---------+------------+
| age(42) | 0.0243902  |
+---------+------------+
| age(43) | 0.0256098  |
+---------+------------+
| age(44) | 0.0390244  |
+---------+------------+
| age(45) | 0.0243902  |
+---------+------------+
| age(46) | 0.0243902  |
+---------+------------+
| age(47) | 0.0146341  |
+---------+------------+
| age(48) | 0.0243902  |
+---------+------------+
| age(49) | 0.0170732  |
+---------+------------+
| age(50) | 0.0195122  |
+---------+------------+
| age(51) | 0.0414634  |
+---------+------------+
| age(52) | 0.0426829  |
+---------+------------+
| age(53) | 0.0280488  |


In [17]:
print(model.get_cpds('chol'))

+-----------+-----------------------+----------------------+
| target    | target(0)             | target(1)            |
+-----------+-----------------------+----------------------+
| chol(126) | 0.0                   | 0.004694835680751174 |
+-----------+-----------------------+----------------------+
| chol(131) | 0.005076142131979695  | 0.0                  |
+-----------+-----------------------+----------------------+
| chol(141) | 0.0                   | 0.007042253521126761 |
+-----------+-----------------------+----------------------+
| chol(149) | 0.005076142131979695  | 0.009389671361502348 |
+-----------+-----------------------+----------------------+
| chol(157) | 0.0                   | 0.009389671361502348 |
+-----------+-----------------------+----------------------+
| chol(160) | 0.0                   | 0.007042253521126761 |
+-----------+-----------------------+----------------------+
| chol(164) | 0.007614213197969543  | 0.0                  |
+-----------+-----------

In [18]:
print(model.get_cpds('sex'))

+--------+----------+
| sex(0) | 0.317073 |
+--------+----------+
| sex(1) | 0.682927 |
+--------+----------+


In [19]:
from pgmpy.inference import VariableElimination
target_infer = VariableElimination(model)
q = target_infer.query(variables=['target'], evidence={'age': 29, 'age':39,  'age':49, 'age':59 })
print(q)

+-----------+---------------+
| target    |   phi(target) |
+===========+===============+
| target(0) |        0.5003 |
+-----------+---------------+
| target(1) |        0.4997 |
+-----------+---------------+


In [20]:
q = target_infer.query(variables=['target'], evidence={'chol': 330, 'chol': 335})
print(q)

+-----------+---------------+
| target    |   phi(target) |
+===========+===============+
| target(0) |        1.0000 |
+-----------+---------------+
| target(1) |        0.0000 |
+-----------+---------------+
